In [1]:
conda install -c conda-forge gdown

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gdown-3.12.2               |     pyh9f0ad1d_0          12 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          12 KB

The following NEW packages will be INSTALLED:

  gdown              conda-forge/noarch::gdown-3.12.2-pyh9f0ad1d_0



gdown-3.12.2         | 12 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gdown
url = 'https://drive.google.com/uc?id=1rcz0ZRJ0uLoEucq8mposcsUOqFn202bf'
output = 'file.mat'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1rcz0ZRJ0uLoEucq8mposcsUOqFn202bf
To: /kaggle/working/file.mat
274MB [00:02, 110MB/s]  


'file.mat'

In [3]:
from scipy.io import loadmat
annots = loadmat('file.mat')
data = [item[0:] for item in annots['h']]

In [4]:
import random
import os
import tensorflow as tf
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Conv1D
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Embedding, BatchNormalization, LayerNormalization
import tensorflow_addons as tfa
from keras.layers import Concatenate, LeakyReLU
from keras.layers import Reshape
from keras.utils.vis_utils import plot_model
from numpy.random import randint
import numpy as np
import time
#from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier

In [5]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
seed_everything(42)

In [6]:
data_array = np.array(data)
data_env1 = data_array[:, :, 0]
dataset = []
num_angles = 90
angle = 0
for angle in range(num_angles):
    angle_i_data = data_env1[:, angle]
    dataset.append(angle_i_data)
dataset = np.array(dataset)

In [7]:
def define_discriminator(latent_dim,n_classes = 10):
    # label input
    in_label = Input(shape=(1,))
    #embedding for categorical input
    li = Embedding(n_classes , 25)(in_label)
    #linear multiplication
    n_nodes = 1
    li = Dense(n_nodes)(li)
    li = Reshape((n_nodes, 1))(li)
    in_dim = Input(shape=(latent_dim, 1))
    merge = Concatenate()([in_dim,li])
    merge = Conv1D(filters=256, kernel_size=5, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=128, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=64, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=64, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=64, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(64, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=16, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(100, activation='relu')(merge)
    out_layer = Dense(1, activation='sigmoid')(merge)
    model = Model([in_dim, in_label], out_layer)
    loss = tf.keras.losses.BinaryCrossentropy()
    #model.compile(loss='mean_absolute_percentage_error', optimizer=tf.keras.optimizers.Adam(lr=1e-4, beta_1 = 0.901), metrics=['accuracy'])
    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1 = 0.901), metrics=['accuracy'])
    return model

In [8]:
def define_generator(latent_dim_gen, n_classes = 10, n_outputs = 1):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes ,25)(in_label)
    # linear multiplication
    n_nodes = 15
    li = Dense(n_nodes)(li)
    # image generator input
    in_lat = Input(shape=(latent_dim_gen,))
    gen = Dense(n_nodes, activation = 'relu',  kernel_initializer='he_uniform')(in_lat)
    gen = Reshape((1,15))(gen)
    merge = Concatenate()([gen,li])
    merge = Conv1D(filters=256, kernel_size=5, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(32, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=128, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(32, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=64, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    merge = Dense(32, activation = 'relu',  kernel_initializer='he_uniform')(merge)
    merge = Conv1D(filters=16, kernel_size=3, padding='same')(merge)
    merge = BatchNormalization()(merge, training=True)
    merge = tf.nn.relu(merge)
    out_layer = Dense(n_outputs, activation='linear')(merge)
    model = Model([in_lat, in_label], out_layer)
    return model

In [9]:
def GAN_model(generator, discriminator):
    discriminator.trainable = False
    gen_noise, gen_label = generator.input
    gen_output = generator.output
    gan_output = discriminator([gen_output, gen_label])
    model = Model([gen_noise, gen_label], gan_output)
    #model.compile(loss='mean_absolute_percentage_error', optimizer=tf.keras.optimizers.Adam(lr=1e-4, beta_1 = 0.901), metrics=['accuracy'])
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.7015)
    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1 = 0.901), metrics=['accuracy'])
    return model

In [10]:
# X = []
# y = []
# angle_data = []
# angle_num = []
# for i in range(90):
#     angle_data = np.append(angle_data, dataset[i])
#     angle_num = np.append(angle_num, [i]*100000)
# angle_num = angle_num.astype(int)

In [11]:
angle_sets = []
angle_set_num = []
num_gans = 10
total_angles = 9

for i in range(0, len(dataset), 10):
    angle_slice = dataset[i:i + 10]
    angle_sets = np.append(angle_sets, angle_slice)
    #angle_set_num.append([[i+j]*100000 for j in range(10)])
    for j in range(0, 10):
        angle_set_num = np.append(angle_set_num, [i+j]*100000)
    
#angle_sets = np.array(angle_sets)
#angle_set_num = angle_set_num.astype(int)
print(angle_sets.shape)
print(angle_set_num.shape)

(9000000,)
(9000000,)


In [12]:
print(angle_set_num[1000000:2000000])

[10. 10. 10. ... 19. 19. 19.]


In [13]:
#n = len(dataset)
import random
def generate_real_samples(angle_data, angle_num, n_samples, n_gan):
	# choose random instances
	ix = randint(0, angle_data.shape[0], n_samples)
	# select images and labels
	angle_data, angle_num = angle_data[ix], angle_num[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [angle_data, angle_num], y

In [27]:
#Generate points in latent space as input for the generator
def generate_latent_points(latent_dim_gen, n, n_gan, n_classes = 9):
    # generate points in the latent space
    x_input = randn(latent_dim_gen * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim_gen)
    labels = randint(int((n_gan-1)*10), int(n_gan*10) - 1, n)
    print(x_input, labels)
    return [x_input, labels]

In [15]:
def generate_fake_samples(generator, latent_dim_gen, n, n_gan):
        # generate points in latent space
        data, angles = generate_latent_points(latent_dim_gen, n, n_gan)
        #print(data.shape)
        #print(angles.shape)
        # predict outputs
        X = generator.predict([data, angles])
        # create class labels
        y = zeros((n, 1))
        return [X,angles], y

In [22]:
def train(g_model, d_model, gan_model, latent_dim, latent_dim_gen, angle_data, angle_num, n_gan, n_epochs=430, n_batch=256):
        # determine half the size of one batch, for updating the discriminator
        half_batch = int(n_batch / 2)
        # manually enumerate epochs
        for i in range(n_epochs):
                # prepare real samples
                [x_real,angle_real], y_real = generate_real_samples(angle_data, angle_num, half_batch, n_gan)
                # prepare fake examples
                [x_fake,angle_fake], y_fake = generate_fake_samples(g_model, latent_dim_gen, half_batch, n_gan)
                # update discriminator
                d_model.train_on_batch([x_real, angle_real], y_real)
                d_model.train_on_batch([x_fake, angle_fake], y_fake)
                # prepare points in latent space as input for the generator
                [x_input, angle_input] = generate_latent_points(latent_dim_gen, n_batch, n_gan)
                # create inverted labels for the fake samples
                y_gan = ones((n_batch, 1))
                # update the generator via the discriminator's error
                gan_model.train_on_batch([x_input, angle_input], y_gan)
                # evaluate the model every n_eval epochs
        gan_model.save('cgan_generator.h')  

In [17]:
TL = 1000000

set1 = angle_sets[:TL]
set2 = angle_sets[TL:2*TL]
set3 = angle_sets[2*TL:3*TL]
set4 = angle_sets[3*TL:4*TL]
set5 = angle_sets[4*TL:5*TL]
set6 = angle_sets[5*TL:6*TL]
set7 = angle_sets[6*TL:7*TL]
set8 = angle_sets[7*TL:8*TL]
set9 = angle_sets[8*TL:9*TL]

In [18]:
angle1 = angle_set_num[:TL]
angle2 = angle_set_num[TL:2*TL]
angle3 = angle_set_num[2*TL:3*TL]
angle4 = angle_set_num[3*TL:4*TL]
angle5 = angle_set_num[4*TL:5*TL]
angle6 = angle_set_num[5*TL:6*TL]
angle7 = angle_set_num[6*TL:7*TL]
angle8 = angle_set_num[7*TL:8*TL]
angle9 = angle_set_num[8*TL:9*TL]

In [23]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator1 = define_discriminator(latent_dim)
# create the generator
generator1 = define_generator(latent_dim_gen)
# create the gan
gan_model1 = GAN_model(generator1, discriminator1)
# train model
train(generator1, discriminator1, gan_model1, latent_dim, latent_dim_gen, set1, angle1, 1)
generator1.save_weights(f'./generator-1')

In [28]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator2 = define_discriminator(latent_dim)
# create the generator
generator2 = define_generator(latent_dim_gen)
# create the gan
gan_model2 = GAN_model(generator2, discriminator2)
# train model
train(generator2, discriminator2, gan_model2, latent_dim, latent_dim_gen, set2, angle2, 2)
generator2.save_weights(f'./generator-2')

[13 10 11 16 17 18 10 16 10 18 18 11 11 18 14 15 18 14 16 17 13 15 15 10
 16 14 18 14 12 16 16 14 12 10 16 16 16 10 15 13 18 14 15 15 13 18 10 12
 10 14 12 13 11 10 11 14 18 16 13 18 15 13 15 12 12 10 10 11 12 10 10 16
 12 11 16 13 14 10 17 12 16 16 11 11 17 17 10 15 15 17 14 14 13 14 11 17
 12 11 11 12 12 14 12 12 15 11 14 14 16 12 17 12 12 11 13 14 12 18 11 13
 17 18 13 15 18 16 14 17]


InvalidArgumentError:  indices[0,0] = 13 is not in [0, 10)
	 [[node functional_39/embedding_13/embedding_lookup (defined at <ipython-input-15-3ebeb50ac35d>:7) ]] [Op:__inference_predict_function_85007]

Errors may have originated from an input operation.
Input Source operations connected to node functional_39/embedding_13/embedding_lookup:
 functional_39/embedding_13/embedding_lookup/84752 (defined at /opt/conda/lib/python3.7/contextlib.py:112)

Function call stack:
predict_function


In [ ]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator3 = define_discriminator(latent_dim)
# create the generator
generator3 = define_generator(latent_dim_gen)
# create the gan
gan_model3 = GAN_model(generator3, discriminator3)
# train model
train(generator3, discriminator3, gan_model3, latent_dim, latent_dim_gen, set3, angle3, 3)
generator3.save_weights(f'./generator-3')

In [ ]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator4 = define_discriminator(latent_dim)
# create the generator
generator4 = define_generator(latent_dim_gen)
# create the gan
gan_model4 = GAN_model(generator4, discriminator4)
# train model
train(generator4, discriminator4, gan_model4, latent_dim, latent_dim_gen, set4, angle4, 4)
generator4.save_weights(f'./generator-4')

In [ ]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator5 = define_discriminator(latent_dim)
# create the generator
generator5 = define_generator(latent_dim_gen)
# create the gan
gan_model5 = GAN_model(generator5, discriminator5)
# train model
train(generator5, discriminator5, gan_model5, latent_dim, latent_dim_gen, set5, angle5, 5)
generator5.save_weights(f'./generator-5')

In [25]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator6 = define_discriminator(latent_dim)
# create the generator
generator6 = define_generator(latent_dim_gen)
# create the gan
gan_model6 = GAN_model(generator6, discriminator6)
# train model
train(generator6, discriminator6, gan_model6, latent_dim, latent_dim_gen, set6, angle6, 6)
generator6.save_weights(f'./generator-6')

InvalidArgumentError:  indices[0,0] = 58 is not in [0, 10)
	 [[node functional_27/embedding_9/embedding_lookup (defined at <ipython-input-15-3ebeb50ac35d>:7) ]] [Op:__inference_predict_function_80503]

Errors may have originated from an input operation.
Input Source operations connected to node functional_27/embedding_9/embedding_lookup:
 functional_27/embedding_9/embedding_lookup/80248 (defined at /opt/conda/lib/python3.7/contextlib.py:112)

Function call stack:
predict_function


In [26]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator7 = define_discriminator(latent_dim)
# create the generator
generator7 = define_generator(latent_dim_gen)
# create the gan
gan_model7 = GAN_model(generator7, discriminator7)
# train model
train(generator7, discriminator7, gan_model7, latent_dim, latent_dim_gen, set7, angle7, 7)
generator7.save_weights(f'./generator-7')

InvalidArgumentError:  indices[0,0] = 67 is not in [0, 10)
	 [[node functional_33/embedding_11/embedding_lookup (defined at <ipython-input-15-3ebeb50ac35d>:7) ]] [Op:__inference_predict_function_82755]

Errors may have originated from an input operation.
Input Source operations connected to node functional_33/embedding_11/embedding_lookup:
 functional_33/embedding_11/embedding_lookup/82500 (defined at /opt/conda/lib/python3.7/contextlib.py:112)

Function call stack:
predict_function


In [ ]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator8 = define_discriminator(latent_dim)
# create the generator
generator8 = define_generator(latent_dim_gen)
# create the gan
gan_model8 = GAN_model(generator8, discriminator8)
# train model
train(generator8, discriminator8, gan_model8, latent_dim, latent_dim_gen, set8, angle8, 8)
generator8.save_weights(f'./generator-8')

In [ ]:
latent_dim = 1
latent_dim_gen = 1
# create the discriminator
discriminator9 = define_discriminator(latent_dim)
# create the generator
generator9 = define_generator(latent_dim_gen)
# create the gan
gan_model9 = GAN_model(generator9, discriminator9)
# train model
train(generator9, discriminator9, gan_model9, latent_dim, latent_dim_gen, set9, angle9, 9)
generator9.save_weights(f'./generator-9')

In [ ]:
i = 4 #choosing angle 4
angle_i_data = dataset[i]
data_hist = np.array(angle_i_data)
data_hist.shape
plt.hist(data_hist, bins=300)
plt.title('Column 85 - Env3 - Real')
plt.savefig('Real-85-3.png')
plt.show()

In [ ]:
def generate_sample_space(latent_dim_gen, angle, n = 100000, n_classes = 10):
    # generate points in the latent space
    x_input = randn(latent_dim_gen * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim_gen)
    labels = [angle] * n
    labels = np.array(labels)
    return [x_input, labels]

In [ ]:
def generate_samples(generator, angle, latent_dim = 1, n=100000):
    latent_pts, labels = generate_sample_space(latent_dim, angle)
    #print(latent_pts.shape)
    #print(labels.shape)
    X = generator.predict([latent_pts, labels])
    y = zeros((n, 1))
    return X, y

In [ ]:
x_fake, _ = generate_samples(generator, 4)
x_fake = x_fake.reshape(100000,1)
plt.hist(x_fake, bins=300)
plt.title('Column 85 - Env3 - Generated')
plt.savefig('Gen-85-3.png')
plt.show()

In [ ]:
print("--------------------------- Generator 1 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(10):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator1, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
print("--------------------------- Generator 2 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(10, 20):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator2, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
print("--------------------------- Generator 3 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(20, 30):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator3, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
print("--------------------------- Generator 4 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(30, 40):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator4, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
print("--------------------------- Generator 5 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(40, 50):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator5, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
print("--------------------------- Generator 6 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(50, 60):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator6, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
print("--------------------------- Generator 7 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(60, 70):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator7, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
print("--------------------------- Generator 8 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(70, 80):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator8, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
print("--------------------------- Generator 9 ---------------------------------")
variance_errors = []
mean_errors = []
for angle in range(80, 90):
    angle_i_data = dataset[angle]
    data_hist = np.array(angle_i_data)
    x_fake, _ = generate_samples(generator9, angle)
    print('='*10+f'angle-{angle+1}'+'='*10)
    real_mean = np.mean(data_hist)
    real_variance = np.std(data_hist)**2
    fake_mean = np.mean(x_fake)
    fake_variance = np.std(x_fake)**2
    mean_error = np.abs(real_mean - fake_mean)
    variance_error = np.abs(real_variance - fake_variance)
    mean_errors.append(mean_error)
    variance_errors.append(variance_error)
    print(f'Real_Mean:{real_mean}; Real_Variance:{real_variance}')
    print(f'Fake_Mean:{fake_mean}; Fake_Variance:{fake_variance}')
    print(f'Mean_Error:{mean_error}; Variance_Error:{variance_error}')
    
print('-'*20)    
print(f'Final_Mean_Error:{np.mean(mean_errors)}; Final_Variance_Error:{np.mean(variance_errors)}')    

In [ ]:
# for angle in range(90):
#     x_fake, _ = generate_samples(generator, angle)
#     x_fake = x_fake.reshape(100000,1)
#     plt.subplot(1,2,1)
#     plt.hist(x_fake, bins=300)
#     print(f'Angle:{angle}, Generated') 
#     angle_i_data = dataset[angle]
#     data_hist = np.array(angle_i_data)
#     data_hist.shape
#     plt.subplot(1,2,2)
#     plt.hist(data_hist, bins=300)
#     print(f'Angle:{angle}, Real') 
#     plt.show()